# One-layer MOSAIKS

## Load utils

In [1]:
import os
from pathlib import Path
import torch
import torch.nn as nn

/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/ffcv2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define the model: two-layer CNN

In [2]:
# load the 2-layer mosaiks model
# mosaiks+kmeans: 
# two_layer_mosaiks_kmeans_7_no_whiten_zcaBias_0.001
# two_layer_mosaiks_kmeans_7_whiten_allGeo_bs32_25eachimage_zcaBias_0.001.pt
# original mosaiks:
# two_layer_mosaiks_original_7_whiten_zcaBias_0.001
# two_layer_mosaiks_original_7_no_whiten_zcaBias_0.001
main_dir = "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/ckpts"
model_name = "two_layer_mosaiks_kmeans_7_whiten_allGeo_bs32_25eachimage_zcaBias_0.001.pt" 
num_filters = [100, 64]
num_species = 17037
# params to save the one-layer model
model_type = "one_layer_mosaiks_kmeans"
mode = "whiten"
kernel_size = 7
zca_bias = 0.001

In [3]:
model = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=num_filters[0], kernel_size=kernel_size, padding='same', bias=True),
      nn.LeakyReLU(),
      nn.MaxPool2d(2, stride=2),

      nn.Conv2d(in_channels=num_filters[0], out_channels=num_filters[1], kernel_size=kernel_size, padding='same', bias=True),
      nn.LeakyReLU(),
      nn.MaxPool2d(2, stride=2),
    
      nn.AdaptiveAvgPool2d(9),
    
      nn.Flatten(),
      nn.Dropout(0.5),
      nn.Linear(5184, 512), #50176
      nn.ReLU(),
      nn.Linear(512, num_species)
      ) 
model(torch.rand((1, 3, 224, 224))).shape
model_path = os.path.join(main_dir, model_name)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

# Get the first conv layer

In [4]:
filter_np = model[0].weight.data.numpy().reshape(100,-1)

In [5]:
one_layer_model = model[0]

In [6]:
type(one_layer_model)

torch.nn.modules.conv.Conv2d

In [7]:
model_path = os.path.join(main_dir, f"{model_type}_{kernel_size}_{mode}_zcaBias_{zca_bias}.pt")
torch.save(one_layer_model.state_dict(), model_path)